In [20]:
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

# Initialize the tokenizer and model from Hugging Face
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Helper function to get embeddings using BERT
def get_embedding(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Get the BERT embeddings (we use the last hidden state)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the token embeddings as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Step 1: Define documents
documents = {
    "doc1": "What is the return policy for our products?",
    "doc2": "How can customers track their orders?",
    "doc3": "What payment methods are accepted?"
}

# Step 2: Generate embeddings for each document
document_embeddings = []
metadata = []
for doc_id, content in documents.items():
    embedding = get_embedding(content)
    document_embeddings.append(embedding)
    metadata.append({"id": doc_id, "text": content})

# Convert embeddings to a NumPy array for vector search
embedding_matrix = np.array(document_embeddings)

# Step 3: Querying the system
def query_qa_bot(query):
    query_embedding = get_embedding(query)
    
    # Compute cosine similarities between the query and document embeddings
    similarities = np.dot(embedding_matrix, query_embedding) / (np.linalg.norm(embedding_matrix, axis=1) * np.linalg.norm(query_embedding))
    
    # Get the indices of the top 3 most similar documents
    top_k_indices = similarities.argsort()[-3:][::-1]
    
    # Retrieve matching contexts
    contexts = [metadata[idx]["text"] for idx in top_k_indices]
    combined_context = " ".join(contexts)
    
    # Simulate a response based on the context
    response = f"Based on the context: {combined_context}, here is an answer to your query."
    return response

# Test the QA bot with a user query
user_query = "What payment options do you offer?"
bot_response = query_qa_bot(user_query)
print("Bot Response:", bot_response)


c:\Users\maury\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maury\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Bot Response: Based on the context: What payment methods are accepted? What is the return policy for our products? How can customers track their orders?, here is an answer to your query.
